In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tslearn.generators
import tslearn.preprocessing
import tslearn.metrics

import fastdtw

np.random.seed(42)

# https://pypi.org/project/dtw-python/

* Sensibilité au scaling, au shift, à la présence de trend, voit à la fois les différences de niveau (value) et de différence (behavior) ?
* Dès que la mesure de similarité fait intervenir une somme, on perd la notion d'ordre. Ex: DTW, mesure approximant linéairement par morceau et sommant les carrés des coef de chacun des modèles linéaires: on perd l'ordre dans lequel ils sont et des TS très différentes peuvent avoir une même valeur de mesure.
* Problème des tailles différentes des TS
* DTW+CORRT

http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_769.pdf
http://ama.liglab.fr/~douzal/teaching.html
http://ama.liglab.fr/~douzal/documents/Basics-TS-Metrics-Oct-2016.pdf
https://lig-membres.imag.fr/bisson/cours/M2INFO-AIW-ML/
https://books.google.fr/books?id=jej7MnXLA3AC&pg=PA139&lpg=PA139&dq=Douzal+Chouakria+cort&source=bl&ots=5DWydi2wAo&sig=ACfU3U3QVLukRmv99JhsKboXp5I1-akQgA&hl=fr&sa=X&ved=2ahUKEwj9n6Gy8ZbnAhXFAWMBHUGsCPEQ6AEwCnoECAoQAQ#v=onepage&q=Douzal%20Chouakria%20cort&f=false
file:///C:/Users/t053351/Downloads/Disertacija.pdf
file:///C:/Users/t053351/Downloads/v62i01.pdf

Brief correlation
https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

https://marcocuturi.net/Teaching/KAIST/kaist_2013.pdf
https://marcocuturi.net/Teaching/KU/2013/PRA/Lec2.pdf
https://cs.nyu.edu/~mohri/aml15/thanos_rodrigo_fast_global_alignment.pdf

In [ ]:
dataset = tslearn.generators.random_walks(n_ts=2, sz=30, d=1, random_state=40)
scaler = tslearn.preprocessing.TimeSeriesScalerMeanVariance(mu=0.0, std=1.0)
dataset_scaled = scaler.fit_transform(dataset)

path, sim = tslearn.metrics.dtw_path(dataset_scaled[0], dataset_scaled[1], global_constraint='sakoe_chiba', sakoe_chiba_radius=3)

Scaling nécéssaire car sinon biaise le DTW ? Obligatoire si les différences d'échelles ne rentrent pas dans le critère de différence de forme.

In [ ]:
plt.close('all')
plt.plot(dataset[0], color='r')
plt.plot(dataset[1], color='b')

plt.title('Unscaled time series');

In [ ]:
plt.close('all')
plt.plot(dataset_scaled[0], color='r')
plt.plot(dataset_scaled[1], color='b')
plt.title('Scaled time series');

In [ ]:
plt.close('all')
plt.plot(dataset_scaled[0], color='r')
plt.plot(dataset_scaled[1], color='b')

for i in range(len(dataset[0])):
    plt.plot((i, i), (dataset_scaled[0][i], dataset_scaled[1][i]), color='k', linewidth=0.5, linestyle='--')

plt.title('Scaled time series - Euclidian distance');

In [ ]:
plt.close('all')
plt.plot(dataset_scaled[0], color='r')
plt.plot(dataset_scaled[1], color='b')

for s1_index, s2_index in path:
    plt.plot((s1_index, s2_index), (dataset_scaled[0][s1_index], dataset_scaled[1][s2_index]), color='k', linewidth=0.5, linestyle='--')

plt.title('Scaled time series - DTW mapping');

In [ ]:
plt.close('all')

warped_serie_1 = [dataset_scaled[0][i] for i, _ in path]
warped_serie_2 = [dataset_scaled[1][j] for _, j in path]

plt.plot(warped_serie_1, color='r')
plt.plot(warped_serie_2, color='b')

plt.title('Warped (scaled) time series');

DTW is the euclidian distance between the two time series warped using the optimal warping path.

In [ ]:
plt.close('all')

plt.plot(warped_serie_1, color='r')
plt.plot(warped_serie_2, color='b')

for i in range(len(warped_serie_1)):
    plt.plot((i, i), (warped_serie_1[i], warped_serie_2[i]), color='k', linewidth=0.5, linestyle='--')

plt.title('Warped (scaled) time series - Euclidian distance');

In [ ]:
sim

In [ ]:
np.sqrt(np.sum([(dataset_scaled[0][s1_index]-dataset_scaled[1][s2_index])**2 for s1_index, s2_index in path]))

In [ ]:
serie_1 = [1,1,1,1,2,3,2,1,1]
serie_2 = [0,0,1,2,1,0,1,0,0]

In [ ]:
plt.close('all')
plt.plot(serie_1, color='r')
plt.plot(serie_2, color='b')
plt.title('Unscaled time series');

In [ ]:
serie_1 = (serie_1 - np.mean(serie_1))/np.std(serie_1)
serie_2 = (serie_2 - np.mean(serie_2))/np.std(serie_2)

path, sim = tslearn.metrics.dtw_path(serie_1, serie_2)

In [ ]:
plt.close('all')
plt.plot(serie_1, color='r')
plt.plot(serie_2, color='b')
plt.title('Scaled time series');

In [ ]:
aligned_serie1 = [serie_1[i] for i, _ in path]
aligned_serie2 = [serie_2[j] for _, j in path]

In [ ]:
plt.close('all')
plt.plot(aligned_serie1, color='r')
plt.plot(aligned_serie2, color='b');

In [ ]:
corrt(serie_1, serie_2)

In [ ]:
corrt(aligned_serie1, aligned_serie2)

In [ ]:
plt.close('all')
plt.plot(serie_1, color='r')
plt.plot(serie_2, color='b')

for s1_index, s2_index in path:
    plt.plot((s1_index, s2_index), (serie_1[s1_index], serie_2[s2_index]), color='k', linewidth=0.5, linestyle='--')

plt.title('Scaled time series with DTW mapping');

In [ ]:
def corrt(x, y):
    diff_x = np.diff(x, n=1)
    diff_y = np.diff(y, n=1)
    return np.sum(diff_x * diff_y)/(np.sqrt(np.sum(diff_x ** 2))*np.sqrt(np.sum(diff_y ** 2)))

In [ ]:
corrt(arr_a, arr_b)